In [39]:
from openai import OpenAI
import os
import json
import pandas as pd
from IPython.display import display_markdown

client = OpenAI()

# Set parameters

name = "1"
model = "gpt-3.5-turbo"               # important options: "gpt-4o-mini", "gpt-4o", "gpt-3.5-turbo"
loops = 3
memory_limit = False
memory = 1                            # only in case of limited memory
stable_system_message = False  
assistant_transform_type = "user-assistant"     # only in case of assistant transform, options: "user", "user-assistant", "system"
stream = False

first_prompt = [
  #{"role": "system", "content": ""},
  {"role": "user", "content": ""}
]

# Choose or create the right folder with json files

folder_name = f"{model}"
if memory_limit:
  folder_name += f" memory-{memory}"
if assistant_transform_type is not None:
  folder_name += f" {assistant_transform_type}"
os.makedirs(folder_name, exist_ok=True)

for loop in range(loops):

  # Load completions from json, if continuing with a file generated in the past

  try:
    with open(f"{folder_name}/{name}.json", "r") as json_file:
      completions=json.load(json_file)

  # If not, create a new completions database

  except FileNotFoundError:
    completions = []

  # Collect messages generated so far

  messages = first_prompt
  for completion in completions:
      message = completion["choices"][0]["message"]
      messages.append({"role": message["role"], "content": message["content"]}   
      )

  # Pick prompt messages

  if (memory >= len(messages)) or not memory_limit:
      prompt_messages = messages
  else:
      #print(f"Memory smaller than the entire conversation. Forgetting {len(messages)-memory} messages")
      prompt_messages = []
      if stable_system_message:
        prompt_messages.append(first_prompt[0])
        print(prompt_messages)
        print("stable_system_message")
      prompt_messages = prompt_messages+messages[(len(messages)-memory):]

  # Transform assistant role to assistant and user, user, or system roles

  if assistant_transform_type is not None:
    if assistant_transform_type == "user-assistant":
      i = 1
      while i <= len(prompt_messages) and prompt_messages[-i]["role"] != "system":
        if i%2 == 1:
          prompt_messages[-i]["role"] = "user"
        else:
          prompt_messages[-i]["role"] = "assistant"
        i += 1
    else:
      for prompt_message in prompt_messages:
        if prompt_message["role"] == "assistant":
          prompt_message["role"] = assistant_transform_type


  # Send to OpenAI API to get a new completion

  completion = client.chat.completions.create(
        model=model,
        messages = prompt_messages,
        stream = stream,
      )
  print(f"Prompt messages sent to OpenAI API:{prompt_messages}")
  if stream:
    all_completion_data = []
    content_string = ""
    for chunk in completion:
      all_completion_data.append(chunk)
      print(chunk.choices[0].delta.content or "", end="")
      if type(chunk.choices[0].delta.content) == str:
        content_string.join(chunk.choices[0].delta.content)
    completion = all_completion_data[0]
    delta = completion.choices[0].delta
    content = delta.content
    completion.choices[0] = delta
  else:
    print(completion.choices[0].message.content)

  # Update database and save it to the right json file

  completions.append(completion.model_dump())
  with open(f"{folder_name}/{name}.json", "w") as json_file:
      json.dump(completions,json_file)

 

Prompt messages sent to OpenAI API:[{'role': 'assistant', 'content': ''}, {'role': 'user', 'content': 'Hello! How can I assist you today?'}, {'role': 'assistant', 'content': "Hello! I'm here to help with any questions or information you may need. Just let me know how I can assist you."}, {'role': 'user', 'content': 'Thank you! I would like to know more about the history of robotics and how it has evolved over the years. Can you provide me with some insights on this topic?'}, {'role': 'assistant', 'content': "Of course! I'd be happy to provide you with an overview of the history of robotics and its evolution over the years.\n\nThe concept of robotics dates back to ancient times, with early mentions of automated machines found in ancient mythologies and stories. However, the modern field of robotics as we know it today began in the early 20th century with the development of the first industrial robots.\n\nIn 1954, George Devol and Joseph Engelberger created the world's first industrial r

In [38]:
## Display data

# Load the JSON file

file_path = f"{folder_name}/{name}.json"

#file_path = f"gpt-3.5-turbo memory-1 user/2.json"

with open(file_path, 'r') as file:
    data = json.load(file)

# Normalize the JSON data in completions

normalized_data = pd.json_normalize(data, record_path='choices', meta=[
    'id', 'created', 'model', 'object', 'service_tier', 'system_fingerprint', 
    ['usage', 'completion_tokens'], ['usage', 'prompt_tokens'], ['usage', 'total_tokens']], sep='.',
    meta_prefix='', record_prefix='choices.')

# Make more important columns more visible

normalized_data = normalized_data[['choices.message.content', 'model', 
       'usage.completion_tokens', 'usage.prompt_tokens', 'usage.total_tokens', 'id', 
       'choices.finish_reason', 'choices.index', 'choices.logprobs', 
       'choices.message.role', 'choices.message.function_call', 'choices.message.tool_calls', 
       'created', 'object', 'service_tier', 'system_fingerprint'
       ]]

# Display

content_all = normalized_data["choices.message.content"]
last = content_all.iloc[-1]

print(file_path)
print(len(content_all))

for content in content_all:
   display_markdown(content, raw=True)

gpt-3.5-turbo user-assistant/1.json
7


Hello! How can I assist you today?

Hello! I'm here to help with any questions or information you may need. Just let me know how I can assist you.

Thank you! I would like to know more about the history of robotics and how it has evolved over the years. Can you provide me with some insights on this topic?

Of course! I'd be happy to provide you with an overview of the history of robotics and its evolution over the years.

The concept of robotics dates back to ancient times, with early mentions of automated machines found in ancient mythologies and stories. However, the modern field of robotics as we know it today began in the early 20th century with the development of the first industrial robots.

In 1954, George Devol and Joseph Engelberger created the world's first industrial robot called the Unimate. It was used to automate tasks in the automotive industry, marking the beginning of the widespread adoption of robots in manufacturing.

Throughout the years, robotics has evolved significantly with advancements in technology, materials, and AI. Robots have become more sophisticated, versatile, and capable of performing complex tasks in various industries such as healthcare, agriculture, construction, and space exploration.

Today, robotics continues to push the boundaries of innovation with the development of autonomous robots, collaborative robots (cobots), and other cutting-edge technologies. The field of robotics holds immense potential for transforming industries, improving efficiency, and enhancing our daily lives.

Overall, the history of robotics showcases a remarkable journey of innovation, discovery, and progress, with the potential to revolutionize the way we work and interact with machines in the future.

I hope this overview of the history of robotics and its evolution over the years provides you with some insights on this topic. If you have any more questions or would like to learn more about a specific aspect of robotics, feel free to ask!

Thank you for the detailed overview. If I have any more questions or need further information on robotics, I'll be sure to reach out. I appreciate your assistance!

You're very welcome! If you have any more questions in the future or need assistance with anything else, feel free to reach out. I'm here to help. Have a great day!

In [ ]:
completions[4]